In [ ]:
'''
sns能夠 與 環境中的sqs 進行串接 (v)
sns 可以發消息 進入sqs
另外可用Python對sns進行topic 創建 新增內容推送 (v)
'''

In [1]:
# 引用套件
import boto3

# 建立 sqs_client, sqs_resource, sns_client 的連線
# sqs_client
sqs_client = boto3.client(
    'sqs',
    endpoint_url = 'http://cc104.sqs.local:9324',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False)

# sqs_resource
#sqs_resource = boto3.resource(
#    'sqs',
#    endpoint_url = 'http://cc104.sqs.local:9324',
#    region_name = 'dummy_region',
#    aws_access_key_id = 'dummy_access_key',
#    aws_secret_access_key = 'dummy_secret_key',
#    verify = False)

# sns_client
sns_client = boto3.client(
    'sns',
    endpoint_url = 'http://cc104.sns.local:9911',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False)

In [2]:
# 環境建立好後使用的變數名稱
sqs_queue_url = 'http://cc104.sqs.local:9324/queue/cc104'
sqs_queue_arn = "aws-sqs://cc104?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey="
sns_topic_arn = 'arn:aws:sns:us-east-1:123456789012:cc104'

In [ ]:
# 建立一個 sqs queue
sqs_queue = sqs_client.create_queue( QueueName = 'cc104' )
print(sqs_queue)

# 將 queue 的 URL 存成變數
sqs_queue_url = sqs_queue['QueueUrl']
print(sqs_queue_url)

In [ ]:
# 取用 sqs_queue 中的 Attributes
sqs_queue_atttrs = sqs_client.get_queue_attributes(
    QueueUrl = sqs_queue_url,
    AttributeNames = ['All']
)
print(sqs_queue_atttrs['Attributes'])

# 從 Attributes 中取出 QueueArn 的值，之後要給 sns 訂閱
# 這是正統在雲上的方法
#sqs_queue_arn = sqs_queue_atttrs['Attributes']['QueueArn']
#print(sqs_queue_arn)


In [ ]:
# 但是萬惡的 local環境不吃正統的 ARN
# 根據該 image作者的 db.json你會發現註冊的ARN不是AWS正統的ARN而是
sqs_queue_arn = "aws-sqs://cc104?amazonSQSEndpoint=http://cc104.sqs.local:9324&accessKey=&secretKey="

In [ ]:
# 建立一個 sns_topic
sns_topic = sns_client.create_topic( Name = 'cc104' )
print(sns_topic)

# 從 sns_topic 中取出 TopicArn 的值，之後要給 sns 訂閱
sns_topic_arn = sns_topic['TopicArn']
print(sns_topic_arn)

In [ ]:
# 將 SNS 訂閱 SQS 
sns_client.subscribe(
    TopicArn = sns_topic_arn,
    Protocol = 'sqs',
    Endpoint = sqs_queue_arn
)

In [5]:
# SNS Publish Message to SQS
import json
response = sns_client.publish(
    TopicArn = sns_topic_arn,
    #TargetArn = sqs_queue_arn,
    Message = json.dumps({'default':'Hello World!'}),
    Subject = 'Mission18',
    MessageStructure = 'json'
)

#response = sns_client.publish(
#    TopicArn = sns_topic_arn,
#    TargetArn = sqs_queue_arn,
#    Message = 'Hello World!',
#    Subject = 'Mission18'
#)

print(response)

{'MessageId': '7c2f692f-fec5-44ba-9466-512b8995648a', 'ResponseMetadata': {'RequestId': '54d056df-557c-43b7-ba53-24ba5a7081c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'akka-http/10.0.10', 'date': 'Thu, 17 Jan 2019 01:40:41 GMT', 'content-type': 'text/xml; charset=UTF-8', 'content-length': '270'}, 'RetryAttempts': 0}}


In [4]:
# Receive Message from SQS queue
message = sqs_client.receive_message(
    QueueUrl = sqs_queue_url,
    AttributeNames = [
        'SentTimestamp'
    ],
    MaxNumberOfMessages = 1,
    MessageAttributeNames = [
        'All'
    ],
    VisibilityTimeout = 0,
    WaitTimeSeconds = 0
)
print(message)

{'Messages': [{'MessageId': '933fd821-7aa5-4e27-a8d6-ea4d40b8697d', 'ReceiptHandle': '933fd821-7aa5-4e27-a8d6-ea4d40b8697d#608a9947-1ce3-4116-bb8d-7b8bf0b82c7a', 'MD5OfBody': '39cc45771bc3726f14d057de9c9ad735', 'Body': '{"MessageId":"10a46ac0-d6f5-4e1a-a0cd-1fa7af8a8b40","Message":"Hello World!","Type":"Notification"}', 'Attributes': {'SentTimestamp': '1547689201962'}, 'MD5OfMessageAttributes': 'be4a68f0e1b91fcf9166c30df4afad81', 'MessageAttributes': {'x-amz-sns-topic-arn': {'StringValue': 'arn:aws:sns:us-east-1:123456789012:cc104', 'DataType': 'String'}, 'x-amz-sns-message-type': {'StringValue': 'Notification', 'DataType': 'String'}, 'x-amz-sns-subscription-arn': {'StringValue': 'arn:aws:sns:us-east-1:123456789012:cc104:16e6bcac-d058-49d4-9e85-11a5f77828d2', 'DataType': 'String'}}}], 'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'akka-http/10.0.2', 'date': 'Thu, 17 Jan 2019 01:40:11 GMT', 'content-type': 'tex

In [6]:
# Receive Message from SQS queue
while True:
    messages = sqs_client.receive_message(
        QueueUrl = sqs_queue_url,
        #AttributeNames=[
        #    'SentTimestamp'
        #],
        MaxNumberOfMessages=1,
        #MessageAttributeNames=[
        #    'All'
        #],
        #VisibilityTimeout=0,
        WaitTimeSeconds = 20
    )
    if 'Messages' in messages:
        for message in messages['Messages']:
            print(message['Body'])
            sqs_client.delete_message(
                QueueUrl = sqs_queue_url,
                ReceiptHandle = message['ReceiptHandle']
            )
    else:
        print('Queue is now empty')
        break

{"MessageId":"10a46ac0-d6f5-4e1a-a0cd-1fa7af8a8b40","Message":"Hello World!","Type":"Notification"}
{"MessageId":"7c2f692f-fec5-44ba-9466-512b8995648a","Message":"Hello World!","Type":"Notification"}
Queue is now empty


In [ ]:
# Process messages by printing out body and optional author name
#for message in sqs_queue_name.receive_messages(MessageAttributeNames=['Author']):
    # Get the custom author message attribute if it was set
#    author_text = ''
#    if message.message_attributes is not None:
#        author_name = message.message_attributes.get('Author').get('StringValue')
#        if author_name:
#            author_text = ' ({0})'.format(author_name)
#
    # Print out the body and author (if set)
#    print('Hello, {0}!{1}'.format(message.body, author_text))

    # Let the queue know that the message is processed
#    message.delete()

In [ ]:
#response = sns_client.get_subscription_attributes(
#    SubscriptionArn = sqs_queue_arn
#)
#print(response)